In [4]:
import requests
import bs4
import numpy as np
import pandas as pd

In [11]:
result = requests.get("https://wikipedia.org/wiki/List_of_countries_by_GDP_(PPP)_per_capita")
soup = bs4.BeautifulSoup(result.text, "lxml")
table = soup.select('table')
#print(table)
body = table[2].select('tbody')
print(body)
rows = body[0].select('tr')
rows = rows[2:]
rows

[<tbody><tr class="static-row-header" style="text-align:center;vertical-align:bottom;">
<th rowspan="2">Country/Territory
</th>
<th rowspan="2"><a href="/wiki/United_Nations_geoscheme" title="United Nations geoscheme">UN Region</a>
</th>
<th colspan="2"><a href="/wiki/International_Monetary_Fund" title="International Monetary Fund">IMF</a><sup class="reference" id="cite_ref-IMF_6-0"><a href="#cite_note-IMF-6">[5]</a></sup><sup class="reference" id="cite_ref-7"><a href="#cite_note-7">[6]</a></sup>
</th>
<th colspan="2"><a href="/wiki/World_Bank" title="World Bank">World Bank</a><sup class="reference" id="cite_ref-8"><a href="#cite_note-8">[7]</a></sup>
</th>
<th colspan="2"><a href="/wiki/Central_Intelligence_Agency" title="Central Intelligence Agency">CIA</a><sup class="reference" id="cite_ref-9"><a href="#cite_note-9">[8]</a></sup>
</th></tr>
<tr class="static-row-header" style="text-align:center;vertical-align:bottom;">
<th>Estimate
</th>
<th>Year
</th>
<th>Estimate
</th>
<th>Year
</

[<tr class="static-row-header" style="background:#F0E891">
 <td style="text-align:left"><span class="flagicon" style="display:inline-block;width:25px;text-align:left;"><img alt="" class="thumbborder" data-file-height="800" data-file-width="1000" decoding="async" height="15" src="//upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Flag_of_Monaco.svg/19px-Flag_of_Monaco.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Flag_of_Monaco.svg/29px-Flag_of_Monaco.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Flag_of_Monaco.svg/38px-Flag_of_Monaco.svg.png 2x" width="19"/></span> <a href="/wiki/Economy_of_Monaco" title="Economy of Monaco">Monaco *</a></td>
 <td style="text-align:center"><a href="/wiki/Europe" title="Europe">Europe</a></td>
 <td class="table-na" colspan="2" data-sort-value="" style="background: #ececec; color: #2C2C2C; vertical-align: middle; text-align: center;">—</td>
 <td>190,513</td>
 <td>2019</td>
 <td>115,700</td>
 <td>2015
 </td></tr>

In [12]:
rows.pop(116)
countries = np.empty([len(rows),2], dtype=object)
raw_data = np.empty([len(rows),2],np.float64)
raw_data[:] = np.nan

In [18]:
%xmode Verbose
for row in range(len(rows)):

        countries[row,0] = rows[row].select('a')[0].getText()[:-2]
        countries[row,1] = rows[row].select('a')[1].getText()
        
        element = 2
        column = 0

        while(column<2):
            field = str(rows[row].select('td')[element].getText())
            if(field=="—"):
                element+=1
            else:
                try:
                    raw_data[row,column] = np.float64(field.replace(",",""))
                    element+=1
                    column+=1
                except ValueError as e:
                    raw_data[row,column] = np.float64(field.replace(",","")[-4:])
                    print(e.args)
                    element+=1
                    column+=1

Exception reporting mode: Verbose
("could not convert string to float: '[n 2]2022'",)


In [19]:
countries_df = pd.DataFrame(countries)
data_df = pd.DataFrame(raw_data)
all_data = pd.concat((countries_df,data_df),axis=1)

In [22]:
all_data.columns = ['Countries', 'UN_Region', 'GPD_per_capita', 'Year']
all_data

Countries         Palestine
UN_Region              Asia
GPD_per_capita       6243.0
Year                 2022.0
Name: 171, dtype: object

In [23]:
all_data.to_csv('WorldWideGPD')